In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import risktools as rt
from loader import _load_json
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
up = {"m*": {"user": "", "pass": ""}, "eia": "", "quandl": ""}

up["eia"] = os.getenv("EIA")
up["quandl"] = os.getenv("QUANDL")
up["m*"]["pass"] = os.getenv("MS_PASS")
up["m*"]["user"] = os.getenv("MS_USER")


In [5]:
assets = rt.simGBM_MV(s0=[100,100, 100], r=0.01, sigma=[0.1,0.1,0.1], T=1, dt=1/252, cor=[[1,0.5,0.5],[0.5,1,0.5],[0.5,0.5,1]])
assets = rt.calc_payoffs(assets)

In [6]:
weights = rt.generate_random_portfolio_weights(3, 100)

In [8]:
port = rt.sim_efficient_frontier(assets=assets, weights=weights)

In [11]:
rt.make_efficient_frontier_table(port, weights, ['A','B','C'])

,Risk,Expected Return,A,B,C
0,129.831158,25366.951810,0.096321,0.171873,0.731806
1,121.136204,25364.706945,0.204814,0.397682,0.397504
2,128.019895,25366.335332,0.058613,0.264268,0.677118
3,121.030226,25365.554689,0.383890,0.213359,0.402750
4,132.873482,25365.090279,0.793900,0.085620,0.120479
...,...,...,...,...,...
95,121.240558,25364.010877,0.356597,0.414520,0.228883
96,127.935961,25362.551327,0.424353,0.562588,0.013059
97,124.201406,25363.137587,0.359963,0.519846,0.120190
98,127.417729,25363.058221,0.075590,0.657560,0.266850
